In [35]:
def initiate_cluster(cname="JE-Test"):
    import boto3
    from datetime import datetime
    import json
    import sys
    
    sys.path.append("/Users/josepherlanger/github/je_scripts/github_scripts/scratch/local")
    import sconfig
    
    with open('config_jsons/basic_config.json') as conf:
        emrConf = json.load(conf)
        
    with open('config_jsons/basic_bootstrap.json') as boot:
        bootstrapConf = json.load(boot)
        
    with open('config_jsons/basic_app.json') as app:
        appConf = json.load(app)
    
    connection = boto3.client('emr',
                              region_name='us-east-1',
                              aws_access_key_id=sconfig.cph_access_key,
                              aws_secret_access_key=sconfig.cph_secret_key
                             )
    print('Connection to EMR successful')    
    
    print('Initiating cluster...')
    cluster = connection.run_job_flow(Name=cname,
                                        LogUri='s3n://debug-dwh-liveintent-com/emr-debug-logs/%s/' %(cname),
                                        ReleaseLabel='emr-5.20.0',
                                        VisibleToAllUsers=True,
                                        EbsRootVolumeSize=10,
                                        ScaleDownBehavior='TERMINATE_AT_TASK_COMPLETION',
                                        JobFlowRole='EMR_EC2_DefaultRole',
                                        ServiceRole='EMR_DefaultRole',
                                        Tags=[
                                             {
                                                 'Key': 'Name',
                                                 'Value': cname,
                                             },
                                             {
                                                 'Key': 'cost-center',
                                                 'Value': 'JE'
                                             },
                                         ],
                                        Applications=appConf,
                                        Instances={
                                            'InstanceFleets': [
                                                {
                                                    'Name': 'Master',
                                                    'InstanceFleetType': 'MASTER',
                                                    'TargetOnDemandCapacity': 0,
                                                    'TargetSpotCapacity': 1,
                                                    'InstanceTypeConfigs': [
                                                        {
                                                            'InstanceType': 'm3.xlarge',
                                                            'BidPriceAsPercentageOfOnDemandPrice': 100.0,
                                                        }
                                                    ],
                                                    'LaunchSpecifications': {
                                                        'SpotSpecification': {
                                                            'TimeoutDurationMinutes': 60,
                                                            'TimeoutAction': 'TERMINATE_CLUSTER',
                                                        }
                                                    }
                                                },
                                                {
                                                    'Name': 'Core',
                                                    'InstanceFleetType': 'CORE',
                                                    'TargetOnDemandCapacity': 0,
                                                    'TargetSpotCapacity': 1,
                                                    'InstanceTypeConfigs': [
                                                        {
                                                            'InstanceType': 'i2.xlarge',
                                                            'BidPriceAsPercentageOfOnDemandPrice': 100.0,
                                                        }
                                                    ],
                                                    'LaunchSpecifications': {
                                                        'SpotSpecification': {
                                                            'TimeoutDurationMinutes': 60,
                                                            'TimeoutAction': 'TERMINATE_CLUSTER',
                                                        }
                                                    }
                                                }
                                            ],
                                            'Ec2KeyName': 'airflow',
                                            'Ec2SubnetId': 'subnet-badaee95',
                                            'EmrManagedMasterSecurityGroup': 'sg-21078352',
                                            'EmrManagedSlaveSecurityGroup': 'sg-32048041',
                                            'KeepJobFlowAliveWhenNoSteps': True
                                        },
                                         BootstrapActions=bootstrapConf,
                                         Configurations=emrConf
                                        )

    job_flow_id = cluster['JobFlowId']
    print(f'Cluster initiation success: {job_flow_id}')
    
test = initiate_cluster()

Connection to EMR successful
Initiating cluster...
Cluster initiation success: j-PXNZ20SB3X13


In [50]:
class tf(object):
    def __init__(self):
        self.flow = None
    def test_function(self):
        self.flow = 'j-PXNZTest'

test2 = tf()

In [51]:
test2.test_function()

In [52]:
test2.flow

'j-PXNZTest'